# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [2]:
#양자화에 필요한 패키지 설치
# %%capture
# !pip install -U bitsandbytes
# !pip install -U git+https://github.com/huggingface/transformers.git
# !pip install -U git+https://github.com/huggingface/peft.git
# !pip install -U git+https://github.com/huggingface/accelerate.git

## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [3]:
import torch
import warnings
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

In [4]:
%%capture
# ! pip install accelerate

In [5]:
model_id = "kyujinpy/Ko-PlatYi-6B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm(

## 4단계 - 잘 실행되는지 확인

In [7]:
# Define device based on GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the chat template directly in the code
chat_template = "<|startoftext|>{role}: {content}<|endoftext|>"

# Messages to be encoded
messages = [
    {"role": "user", "content": "딥러닝 대해 알려줘"}
]

# Apply the chat template
encodeds = tokenizer.apply_chat_template(messages, chat_template=chat_template, return_tensors="pt")

# Move the encoded inputs to the specified device
model_inputs = encodeds.to(device)

# Generate the response using the model
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

# Decode the generated IDs to text
decoded = tokenizer.batch_decode(generated_ids)

# Print the first decoded response
print(decoded[0])


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|startoftext|> {role}: {content}<|endoftext|>1. 7 oz. water
1⁄2 oz. vodka
1⁄2 oz. limoncello
1⁄2 oz. peach liquer
4 oz. juice sweetened with sugar
1 oz. pineapple juice
Lemon Peel
Peach pieces
Method
Combine water, Vodka, limoncello, peach liquer, and sweetened juice
Add ice
Mix well
Strain into chilled glasses, Garnish with chopped fruit and serve.
7 comments:
Hello! I just stumbled across your blog today and I'm really looking forward to trying your recipes! I've got a question for you : is that a recipe for a daiquiri rather than a limoncello pina colada? (Or was I misreading?) Anyways, I love your blog and I'm really looking forward to following! Check out my blog and perhaps you'll want to try your hand at some of my concoctions, as well! Here's a link to my most recent post:
Yes, it is actually a limoncello Pina Colada. I was about to correct it but since you brought it up I'll change it to Limoncello pina colada. Thanks for pointing that out!
Hey thanks! I didn't even try to ma

## 5단계- RAG 시스템 결합하기

In [8]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
# import warnings
# warnings.filterwarnings('ignore')

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [9]:
# !pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu
# !pip install langchain_community

In [10]:
# !pip install langchain_huggingface

In [11]:
# from langchain.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:
{context}
### QUESTION:
{question}
[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

/tmp/ipykernel_18865/3160757214.py:36: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

### 한 파일로 처리하는 방법

## **핵심정리**

- pdf로 만든다는 것 알겠는데 아래처럼 생긴 데이터를 document object를 어떻게 만드는지 알려줘?
- 엑셀(test)의 정보제공컬럼까지 copy하여 chatGPT에게 물어보고
- Document는 page_content, metadata의 속성을 갖고 그런데 metadata에는 국가명, 수집일자, 제목, 내용, 정보제공국가로 구성된다. 내용이 바로 page_content이다.

- pdf로 만든다는 것 알겠는데 아래처럼 생긴 데이터를 document object를 어떻게 만드는지 알려줘?
- 엑셀(test)의 정보제공컬럼까지 copy하여 chatGPT에게 물어보고
- Document는 page_content, metadata의 속성을 갖고 그런데 metadata에는 국가명, 수집일자, 제목, 내용, 정보제공국가로 구성된다. 내용이 바로 page_content이다.

### **기사를 보고 원인요소를 예측**
#### 1. Data Preparation
- documents list contains both the page_content and the 식품등 유형 in its metadata.
- label each document with the correct "식품등 유형".
##### 1.1. 훈련데이터 준비(2020년 ~ 2022년)
- 전체 엑셀파일 읽어서 피클 gzip을 압축하여 'Combined_Food_Safety_Data_2015_2023_gzip.pkl'로 저장

In [13]:
import pandas as pd
import glob
# Use glob to get all file paths matching the pattern for the years 2020 to 2022
file_paths = glob.glob('/home/ancestor9/python/*.xls')
print(file_paths)

# Initialize an empty list to store dataframes
dfs = []
ttl = 0
# Read each Excel file and append to the list
for file_path in file_paths:
    df = pd.read_excel(file_path)
    df['수집일자'] = pd.to_datetime(df['수집일자'])
    ttl += df.shape[0]
    print(file_path)
    file_name = file_path.split('.')[0]
    print(f'{file_name}: 데이터크기 {df.shape}, 누적크기는 {ttl}')
    dfs.append(df)
# Concatenate all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)
combined_df

pd.to_datetime(df['수집일자'])

# # Save the combined dataframe to a new Excel file
output_file_path = 'Combined_Food_Safety_Data_2015_2023_gzip.pkl'
combined_df.to_pickle(output_file_path, compression='gzip')

print(f"Combined file saved to {output_file_path}")


['/home/ancestor9/python/식품안전정보DB(2021).xls', '/home/ancestor9/python/식품안전정보DB(2022).xls', '/home/ancestor9/python/식품안전정보DB(2020).xls', '/home/ancestor9/python/식품안전정보DB(2023).xls']
/home/ancestor9/python/식품안전정보DB(2021).xls
/home/ancestor9/python/식품안전정보DB(2021): 데이터크기 (23135, 8), 누적크기는 23135
/home/ancestor9/python/식품안전정보DB(2022).xls
/home/ancestor9/python/식품안전정보DB(2022): 데이터크기 (22233, 8), 누적크기는 45368
/home/ancestor9/python/식품안전정보DB(2020).xls
/home/ancestor9/python/식품안전정보DB(2020): 데이터크기 (19607, 8), 누적크기는 64975
/home/ancestor9/python/식품안전정보DB(2023).xls
/home/ancestor9/python/식품안전정보DB(2023): 데이터크기 (20775, 8), 누적크기는 85750
Combined file saved to Combined_Food_Safety_Data_2015_2023_gzip.pkl


In [68]:
import pandas as pd
# Assuming your DataFrame `df` has a column '식품등유형'
file_path = 'Combined_Food_Safety_Data_2015_2023_gzip.pkl'  # Replace with your actual file path
df_all = pd.read_pickle(file_path, compression='gzip')  # Update with the correct file path
for col in df_all.columns:
    if df_all[col].dtype == 'object':
        df_all[col] = df_all[col].astype(str)
df_all.head(2)

,국가명,수집일자,제목,내용,정보제공국가,정보구분,식품등유형,원인요소
0,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제51기)_육류제품","[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 12대 식품(423건 샘플) 중...",중국,위해식품정보,가공식품>식육가공품 및 포장육>양념육류,화학적 위해요소>식품첨가물>삭카린나트륨
1,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제52기)","장시(江西)성 시장감독관리국, 12대 식품(107건 샘플) 중 3건 부적합 <부적합...",중국,위해식품정보,가공식품>가공식품 일반>가공식품 일반,안전위생>위생>위생


In [70]:
df = df_all.copy()
df.head(2)

,국가명,수집일자,제목,내용,정보제공국가,정보구분,식품등유형,원인요소
0,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제51기)_육류제품","[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 12대 식품(423건 샘플) 중...",중국,위해식품정보,가공식품>식육가공품 및 포장육>양념육류,화학적 위해요소>식품첨가물>삭카린나트륨
1,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제52기)","장시(江西)성 시장감독관리국, 12대 식품(107건 샘플) 중 3건 부적합 <부적합...",중국,위해식품정보,가공식품>가공식품 일반>가공식품 일반,안전위생>위생>위생


### Splits a column into multiple columns based on a separator.

In [71]:
def split_column(df, column_name, separator='>'):
    new_columns = df[column_name].str.split(separator, expand=True)
    new_columns.columns = [f"{column_name}_{i+1}" for i in range(new_columns.shape[1])]
    df = pd.concat([df, new_columns], axis=1)
    df.drop(columns=[column_name], inplace=True)
    return df

df_all = split_column(df_all, '원인요소')
df_all = split_column(df_all, '식품등유형')
df_all['수집일자'] = pd.to_datetime(df_all['수집일자'])
df_all = df_all[df_all['수집일자'] <= pd.to_datetime('2023-06-30')]
df_all.head(2)

,국가명,수집일자,제목,내용,정보제공국가,정보구분,원인요소_1,원인요소_2,원인요소_3,식품등유형_1,식품등유형_2,식품등유형_3
0,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제51기)_육류제품","[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 12대 식품(423건 샘플) 중...",중국,위해식품정보,화학적 위해요소,식품첨가물,삭카린나트륨,가공식품,식육가공품 및 포장육,양념육류
1,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제52기)","장시(江西)성 시장감독관리국, 12대 식품(107건 샘플) 중 3건 부적합 <부적합...",중국,위해식품정보,안전위생,위생,위생,가공식품,가공식품 일반,가공식품 일반


#### 훈련데이터는 2015년부터 2023년 6월31일까지

In [72]:
df.head(2)

,국가명,수집일자,제목,내용,정보제공국가,정보구분,식품등유형,원인요소
0,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제51기)_육류제품","[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 12대 식품(423건 샘플) 중...",중국,위해식품정보,가공식품>식육가공품 및 포장육>양념육류,화학적 위해요소>식품첨가물>삭카린나트륨
1,중국,2021-12-31,"중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제52기)","장시(江西)성 시장감독관리국, 12대 식품(107건 샘플) 중 3건 부적합 <부적합...",중국,위해식품정보,가공식품>가공식품 일반>가공식품 일반,안전위생>위생>위생


In [82]:
# Define a custom Document class
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

    def __repr__(self):
        content_preview = str(self.page_content) if pd.notna(self.page_content) else 'No Content'
        return f"Document(page_content={content_preview}..., metadata={self.metadata})"

# Create documents from the DataFrame
documents = [
    Document(
        page_content=row['내용'],
        metadata={
            '국가명': row['국가명'],
            '수집일자': row['수집일자'],
            '제목': row['제목'],
            '원인요소': row['원인요소_2']  # Ensure this column exists in your data
        }
    )
    for _, row in df_all.iterrows()  # df.iterrows() 
]

# Prepare data for training
data = [(doc.page_content, doc.metadata['원인요소']) for doc in documents if '원인요소' in doc.metadata]
df_data = pd.DataFrame(data, columns=['page_content', '원인요소'])

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(df_data['원인요소'].unique())}
df_data['label'] = df_data['원인요소'].map(label_mapping)
df_data

,page_content,원인요소,label
0,"[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 12대 식품(423건 샘플) 중...",식품첨가물,0
1,"장시(江西)성 시장감독관리국, 12대 식품(107건 샘플) 중 3건 부적합 <부적합...",위생,1
2,"구이저우(贵州)성 시장감독관리국, 11대 식품(샘플 919건) 중 27건 부적합(식...",위생,1
3,"[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 10대 식품(564건 샘플) 중...",미생물,2
4,"[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 10대 식품(564건 샘플) 중...",식품첨가물,0
...,...,...,...
74400,-일자 : 2022.12.30.-지역 : 프랑스 전역-제품명 : tartare de...,미생물,2
74401,"[감염현황, 12월 30일 기준] - 발생사례: 15건- 입원자 : 2명- 사망자 ...",미생물,2
74402,-일시: 2022년 12월 29일-제품명: ① Cheese-kama(チーズ蒲) 12...,미생물,2
74403,"- 일자: 2022.12.30- 유통지역: 독일 Berlin, Hamburg, Sc...",이물질,15


In [83]:
df_data.label.nunique()

36

In [84]:
documents[0]

Document(page_content=[중앙 계획 샘플검사]장시(江西)성 시장감독관리국, 12대 식품(423건 샘플) 중 10건 부적합식용농산물 4건, 주류 3건, 외식식품 2건, 육류제품 1건 부적합)*부적합 식용농산물의 생산자 정보 제공하지 않음*주류 3건 부적합은 원문 참조 - 생산기업: 장시(JiangXi:江西) 헤이야팡 식품 유한공사(HeiYaFang:黑鸭坊食品有限公司)- 식품명: 수육오리발(卤鸭掌)- 규격: 160g/봉- 상표: 장펑스헤이야팡(ZhangFengShiHeiYaFang:张冯氏黑鸭坊)- 생산일자/로트번호: 2021년 8월 15일- 부적합항목║검사결과║기준치: 사카린나트륨(사카린으로 계산)（g/kg）║0.104║사용불가..., metadata={'국가명': '중국', '수집일자': Timestamp('2021-12-31 00:00:00'), '제목': '중국 장시성 시장감독관리국, 부적합 식품 상황 통보(2021년 제51기)_육류제품', '원인요소': '식품첨가물'})

In [85]:
isinstance(documents[0].page_content, str)

True

In [86]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

def ensure_string(text):
    if not isinstance(text, str):
        return str(text)
    return text

documents = [doc for doc in documents if isinstance(doc.page_content, str)]

db = FAISS.from_documents(documents, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

In [87]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [88]:
result = rag_chain.invoke("식중독 관련 기사가 몇건이 있었지")

In [89]:
result

{'context': [Document(metadata={'국가명': '일본', '수집일자': Timestamp('2020-06-08 00:00:00'), '제목': '일본 농림수산성, 식중독이 많은 계절은? ', '원인요소': '미생물'}, page_content='일본 농림수산성은 6월 4일, ‘식중독이 많은 계절은?’을 공개함. 주요 내용은 아래와 같음.후생노동성의 통계에 따르면 근래 5년 간의 식중독 평균 발생건수는 변동이 있기는 하지만, 900~1400건의 추이임. 2019년 식중독은 1061건(환자: 13018명)이 보고되고 있음.<식중독의 원인은?>- 2019년 1~12월에 발생한 식중독 발생 원인은 약 40%가 세균, 약 50%가 바이러스였음- 이 때문에 주요 원인인 세균과 바이러스로 인한 식중독 방지 대책을 확실하게 하는 것이 중요함<식중독이 많은 계절은?>- 장마철(5~6월)과 여름(7~9월)은 습도와 기온이 높고, 세균이 증식하기 쉬워 세균성 식중독 발생 건수가 증가하고 있음- 겨울(12~3월)은 노로바이러스 등 바이러스성 식중독의 발생이 보임. 또 봄과 가을에는 다른 시기와 비교해서 자연독으로 인한 식중독이 많이 발생함- 이와 같이 식중독은 연중 일정 발생이 보이기 때문에 평상시부터 식중독 예방을 명심하는 것이 중요함'),
  Document(metadata={'국가명': '미국', '수집일자': Timestamp('2022-04-07 00:00:00'), '제목': "미국 식품의약품청, General Mills(사)의 'Lucky Charms' 시리얼 제품 관련 식중독 조사 중", '원인요소': '미생물'}, page_content='*기수집정보: [식중독]미국, 시리얼 섭취로 인한 식중독 호소자 139명 발생(수집일자: 2022-04-02) 100명 이상의 사람들은 \'iwaspoisoned.com \' 웹사이트에 General Mills(사)의 \'Lucky Charms\' 시리얼 제품이 다양한 위장장애를 유발했다고 주장하는 글을 게

In [90]:
result.keys()

dict_keys(['context', 'question', 'text'])

In [91]:
result = rag_chain.invoke("수집일자가 2023년 3월인 데이터 중에서 '내용'이 식품 위해에 중대한 영향을 미치는  되는 기사 3건을 요약해줘")

for i in result['context']:
    print(f"주어진 근거: {i.metadata['수집일자']} {i.page_content} / 출처: {i.metadata['제목']} - {i.metadata['원인요소']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 2023-03-06 00:00:00 일본 후생노동성은 2023년 3월 수입 시 식품 등의 식품위생법 위반사례를 공개함.-제품명: 신선 참깨(生鮮ゴマの種子)-수출자: ASIAN COMMODITIES CORPORATION.-생산국: 파키스탄-부적합 내용: 아플라톡신(Aflatoxin) 12 μg/kg(B1: 11.6 μg/kg) 검출-조치 내용: 폐기, 반송 등을 지시(전량 보관)-공표일: 2023년 3월 3일-비고: 검사명령 / 출처: 일본 후생노동성, 수입식품등 식품위생법 위반사례(1) - 파키스탄산 신선 참깨(1) - 곰팡이독소 


주어진 근거: 2023-04-03 00:00:00 일본 후생노동성은 2023년 3월 수입 시 식품 등의 식품위생법 위반사례를 공개함.-제품명: 기타 옥수수(GM 불분별)(その他のとうもろこし(GM不分別))-수출자: EVERCORN,INC.-생산국: 미국-부적합 내용: 아플라톡신(Aflatoxin) 12 μg/kg(B1: 12.0 μg/kg) 검출-조치 내용: 폐기, 반송 등을 지시(전량 보관)-공표일: 2023년 3월 31일-비고: 검사명령 / 출처: 일본 후생노동성, 수입식품등 식품위생법 위반사례(4) - 미국산 기타 옥수수(GM 불분별)(2) - 곰팡이독소 


주어진 근거: 2023-03-16 00:00:00 일본 후생노동성은 2023년 3월 수입 시 식품 등의 식품위생법 위반사례를 공개함.-제품명: 신선 참깨(生鮮ゴマの種子)-수출자: AKSHAJ COMMODITIES LIMITED-생산국: 탄자니아-부적합 내용: 성분규격 부적합 - 이미다클로프리드(Imidacloprid) 0.21 ppm 검출-조치 내용: 폐기, 반송 등을 지시(전량 보관)-공표일: 2023년 3월 15일-비고: 검사명령 / 출처: 일본 후생노동성, 수입식품등 식품위생법 위반사례(12) - 탄자니아산 신선 참깨 - 잔류농약 



답변: 
### [INST]
Instruction: Answer the question based on your 

In [92]:
result['text']

"\n### [INST]\nInstruction: Answer the question based on your knowledge.\nHere is context to help:\n[Document(metadata={'국가명': '일본', '수집일자': Timestamp('2023-03-06 00:00:00'), '제목': '일본 후생노동성, 수입식품등 식품위생법 위반사례(1) - 파키스탄산 신선 참깨(1)', '원인요소': '곰팡이독소'}, page_content='일본 후생노동성은 2023년 3월 수입 시 식품 등의 식품위생법 위반사례를 공개함.-제품명: 신선 참깨(生鮮ゴマの種子)-수출자: ASIAN COMMODITIES CORPORATION.-생산국: 파키스탄-부적합 내용: 아플라톡신(Aflatoxin) 12 μg/kg(B1: 11.6 μg/kg) 검출-조치 내용: 폐기, 반송 등을 지시(전량 보관)-공표일: 2023년 3월 3일-비고: 검사명령'), Document(metadata={'국가명': '일본', '수집일자': Timestamp('2023-04-03 00:00:00'), '제목': '일본 후생노동성, 수입식품등 식품위생법 위반사례(4) - 미국산 기타 옥수수(GM 불분별)(2)', '원인요소': '곰팡이독소'}, page_content='일본 후생노동성은 2023년 3월 수입 시 식품 등의 식품위생법 위반사례를 공개함.-제품명: 기타 옥수수(GM 불분별)(その他のとうもろこし(GM不分別))-수출자: EVERCORN,INC.-생산국: 미국-부적합 내용: 아플라톡신(Aflatoxin) 12 μg/kg(B1: 12.0 μg/kg) 검출-조치 내용: 폐기, 반송 등을 지시(전량 보관)-공표일: 2023년 3월 31일-비고: 검사명령'), Document(metadata={'국가명': '일본', '수집일자': Timestamp('2023-03-16 00:00:00'), '제목': '일본 후생노동성, 수입식품등 식품위생법 위반사례(12) - 탄자니아산 신선 

In [93]:
result = rag_chain.invoke("특정 식품등유형에 해당하는 위해기사를 보여줘. (예: 가공식품)")

for i in result['context']:
    print(f"주어진 근거: {i.metadata['수집일자']} {i.page_content} / 출처: {i.metadata['제목']} - {i.metadata['원인요소']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 2023-06-23 00:00:00 위생복지부는 알레르기 체질인 소비자가 안심하고 섭취할 수 있도록 포장식품업체에 식품 알레르기 표시 정보 공개를 강화하도록 이미 요구했음. 포장식품에 갑각류, 망고, 땅콩, 우유 및 양유, 알, 견과류, 참깨, 글루텐 함유 곡물, 대두, 어류 등 10종 및 그 제품이 함유되어 있는 경우, 그리고 아황산염류 등을 사용하여 최종 제품의 이산화황 잔류량이 10mg/kg 이상인 경우, 제품의 용기 또는 외포장에 해당 알레르기 유발 내용물 명칭 등의 주의 정보를 반드시 표시해야 함. 그밖에 위생복지부는 두족류(예:오징어), 권패류(예:우렁이), 종자류(예:해바라기씨), 키위 등 4종 및 그 제품에 대해 주의 문구를 표시하도록 업체에 권고하였음. 식품 생산 제조 과정에서 사용하지 않았으나, 공용 공장, 설비 또는 생산라인에서 알레르기 유발 내용물을 처리한 경우, '본 제품의 생산 제조 공장의 설비 또는 생산라인에서 OO를 처리하고 있습니다.' 또는 동일한 의미의 문구를 표시하도록 권고함. 식품약물관리서는 알레르기 체질인 소비자에 포장 상에 표시된 '본 제품에는 OO이 함유되어 있습니다', '본 제품에는 OO이 함유되어 있어 알레르기 체질인 경우 섭취에 적합하지 않습니다' 또는 동일한 의미의 문구를 꼭 확인할 것을 당부하는 바임. / 출처: 대만 식약서, 포장식품 알레르기 표시 확인 당부 - 알레르기 


주어진 근거: 2021-11-09 00:00:00 -검사 방법: 위생복지부 2020.9.2 위수식자(衛授食字) 제1091901654호 개정 공고 "식품 중 곰팡이독소 검사방법-아플라톡신의 검사(MOHWT0001.04)" -부적합 사유: 아플라톡신B1 14 μg/kg 및 총아플라톡신 16 μg/kg -법정 제한 표준: "식품 중 오염물질 및 독소위생표준"에 의거, 땅콩, 기름종자 및 콩(껍질제거한 원료, 단 유지 정제용으로 공급한 원료는 포함하지 않음)의 아플라톡신B1 및 총아플라톡신의 제한량은 8 μg/kg 및 15 μg/

In [94]:
result.keys()

dict_keys(['context', 'question', 'text'])

In [95]:
result['context']

[Document(metadata={'국가명': '대만', '수집일자': Timestamp('2023-06-23 00:00:00'), '제목': '대만 식약서, 포장식품 알레르기 표시 확인 당부', '원인요소': '알레르기'}, page_content="위생복지부는 알레르기 체질인 소비자가 안심하고 섭취할 수 있도록 포장식품업체에 식품 알레르기 표시 정보 공개를 강화하도록 이미 요구했음. 포장식품에 갑각류, 망고, 땅콩, 우유 및 양유, 알, 견과류, 참깨, 글루텐 함유 곡물, 대두, 어류 등 10종 및 그 제품이 함유되어 있는 경우, 그리고 아황산염류 등을 사용하여 최종 제품의 이산화황 잔류량이 10mg/kg 이상인 경우, 제품의 용기 또는 외포장에 해당 알레르기 유발 내용물 명칭 등의 주의 정보를 반드시 표시해야 함. 그밖에 위생복지부는 두족류(예:오징어), 권패류(예:우렁이), 종자류(예:해바라기씨), 키위 등 4종 및 그 제품에 대해 주의 문구를 표시하도록 업체에 권고하였음. 식품 생산 제조 과정에서 사용하지 않았으나, 공용 공장, 설비 또는 생산라인에서 알레르기 유발 내용물을 처리한 경우, '본 제품의 생산 제조 공장의 설비 또는 생산라인에서 OO를 처리하고 있습니다.' 또는 동일한 의미의 문구를 표시하도록 권고함. 식품약물관리서는 알레르기 체질인 소비자에 포장 상에 표시된 '본 제품에는 OO이 함유되어 있습니다', '본 제품에는 OO이 함유되어 있어 알레르기 체질인 경우 섭취에 적합하지 않습니다' 또는 동일한 의미의 문구를 꼭 확인할 것을 당부하는 바임."),
 Document(metadata={'국가명': '대만', '수집일자': Timestamp('2021-11-09 00:00:00'), '제목': "대만, 인도에서 수출한 '땅콩'에서 곰팡이독소 함량 규정 부적합", '원인요소': '곰팡이독소'}, page_content='-검사 방법: 위생복지부 2020.9.2 위수식자(衛授食字) 제1091901654호 개정 공고 "식품 중 곰팡이독소 검사방법

In [97]:
df_all[(df_all['수집일자'] == '2023-04-03') & (df_all['국가명'] == '일본') & (df_all['원인요소_2'] == '표시광고>알레르기>알레르기일반')]

,국가명,수집일자,제목,내용,정보제공국가,정보구분,원인요소_1,원인요소_2,원인요소_3,식품등유형_1,식품등유형_2,식품등유형_3


In [33]:
result = rag_chain.invoke("식중독은 언제 많이 발생하는가")
for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['제목']} - {i.metadata['원인요소']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 미국 언론 Food Safety News에 따르면, 스웨덴 공중보건청(Folkhälsomyndigheten)은 지난 6월 말과 7월 초  캠필로박터균 식중독 환자가 증가했으며, 신선 닭고기가 오염원으로 유력하다고 발표하였음. 당국은 주당 식중독 환자 수가 6월에는 70명이었지만, 2주 전은 100명, 지난주는 140여명으로 증가했으며, 식중독 증가 추세가 예년보다 갑자기, 그리고 다소 일찍 시작된 것 같다고 설명함.  / 출처: 스웨덴 공중보건청, 최근 캠필로박터균 환자 증가해...오염원으로 신선 닭고기 유력 - 생물학적 위해요소>미생물>캠필로박터 제주니 


주어진 근거: 최근 발표된 데이터에 따르면, 뉴질랜드에서 2022년에 대부분의 식중독 감염이 증가했다고 보고함. 이 보고서(*)는 일차산업부(MPI)의 식품안전부(NZFS)에서 공개하였음. 2022년 27개(253건의 사례) 발생의 대부분은 식품 사업자와 관련이 있었고, 소비자 가정에서 준비한 식품과 관련된 발생은 5건에 불과하였음.  캠필로박터, 살모넬라, 시가독소 생산 대장균(STEC), 리스테리아 감염은 2021년 때보다 2022년 증가하였음.  * 뉴질랜드 일차산업부 '2022년 뉴질랜드 식중독 관련 연간 보고"https://www.mpi.govt.nz/dmsdocument/58789-Annual-report-concerning-Foodborne-Diseases-in-New-Zealand-2022  / 출처: 뉴질랜드 일차산업부, 데이터에 따르면 식중독 감염이 증가해 - 생물학적 위해요소>미생물>미생물일반 


주어진 근거: * 관련 기수집 정보: [식중독]핀란드, 급식으로 제공된 토르티야 섭취 후 200여 명 식중독 (수집일자: 2023-08-19) -일시: 2023. 8. 22.-지역: 핀란드 미켈리(Mikkeli)시-발생 규모 : 600명 이상 (증상 비교적 경미. 입원환자 없음)-발생 사유 : 초기 조사 결과, 야채 토르티야를 제공한 모든 학교에서 복통, 메스꺼움, 두통 등을

### 2. Model Training
- We'll use a simple text classification model. For this example, we'll use transformers from Hugging Face.

In [98]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch

# Define the dataset class
class FoodTypeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_mapping))

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(df_data['page_content'].tolist(), df_data['label'].tolist(), test_size=0.1, random_state=42)

# Create the datasets
train_dataset = FoodTypeDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = FoodTypeDataset(val_texts, val_labels, tokenizer, max_len=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,3.594100
20,3.589700
30,3.558300
40,3.531800
50,3.477800
60,3.333100
70,3.210800
80,2.964500
90,2.916700
100,2.898800


TrainOutput(global_step=6279, training_loss=0.9164522915703606, metrics={'train_runtime': 1851.8604, 'train_samples_per_second': 108.481, 'train_steps_per_second': 3.391, 'total_flos': 1.3218260470419456e+16, 'train_loss': 0.9164522915703606, 'epoch': 3.0})

### 3. Prediction
- To make predictions with the trained model, we need to encode the new text data and use the model to predict the label.

In [99]:
import torch

# Function to predict the "원인요소" for new articles
def predict_food_type(page_content):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    inputs = tokenizer.encode_plus(
        page_content,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()

    # Map the integer prediction back to the label
    label_map = {v: k for k, v in label_mapping.items()}
    predicted_label = label_map[prediction]

    return predicted_label

# Example usage
new_article = '''
호주 언론 'News.com.au'에 따르면, 서호주(West Australia)주 보건당국이 Qukes 브랜드의 오이(baby cucumber) 제품을 섭취한
사람들에게서 설사 증세가 발생하자 해당 제품을 긴급 회수 조치한다고 발표했음. 크리스마스 기간 동안 12명 이상이 해당 오이를 섭취한 후 아팠음.
12월 23일 퍼스(Perth)의 한 슈퍼마켓에서 구입한 오이에서 살모넬라 티피무리움(Salmonella Typhimurium)이 검출되었음.
주 보건당국은 관련 조사에 착수했으며, 2022년 12월에 구입한 제품을 섭취하지 않도록 당부했음.
언론은 다른 브랜드의 오이는 현재까지 영향을 받지 않았다고 보도했음.
'''
predicted_type = predict_food_type(new_article)
print(f"The predicted 원인요소 is: {predicted_type}")

The predicted 원인요소 is: 미생물


### 2020년도 데이터의 원인요소를 예측하기
- 분류유형을 자동으로 구분하여 label이 가능

In [100]:
tf = pd.read_excel('/home/ancestor9/python/식품안전정보DB(2020).xls').sample(n=1000)
tf.head(2)

,국가명,수집일자,제목,내용,정보제공국가,정보구분,식품등유형,원인요소
7613,중국,2020-09-03,"중국 광시장족자치구 시장감독관리국, 식품안전 샘플검사 정보 공고(2020년 제115...","광시장족자치구(广西壮族自治区) 시장감독관리국, 3대 식품(샘플 126건) 중 전분 ...",중국,위해식품정보,가공식품>농산가공식품류>전분류,생물학적 위해요소>미생물>미생물일반
3341,중국,2020-11-09,"중국 장쑤성 시장감독관리국, 부적합 식품 12건 공고(2020년 제35기)_음료(1)","장쑤(江苏)성 시장감독관리국, 식품 샘플 806건 중 12건 부적합(식용농산물 2건...",중국,위해식품정보,환경>생수(먹는샘물)>생수(먹는샘물),생물학적 위해요소>미생물>대장균군


In [101]:
[predict_food_type(text) for text in tf['내용'].str[:1000]]

['미생물',
 '미생물',
 '곰팡이독소',
 '잔류농약',
 '곰팡이독소',
 '잔류농약',
 '위생',
 '동식물질병',
 '기구용기포장유래물질',
 '의약품성분',
 '건강',
 '미생물',
 '방사능',
 '식품첨가물',
 '잔류농약',
 '위생',
 '영양성분',
 '위생',
 '방사능',
 '기타',
 '잔류농약',
 '알레르기',
 '식품첨가물',
 '미생물',
 '식품첨가물',
 '건강',
 '미생물',
 'GMO(LMO)',
 '위생',
 '잔류농약',
 '미생물',
 '식품첨가물',
 '안전',
 '알레르기',
 '영양성분',
 '미생물',
 '동식물질병',
 '곰팡이독소',
 '안전',
 '기구용기포장유래물질',
 '성상',
 '알레르기',
 '미생물',
 '안전',
 '위생',
 '안전',
 '미생물',
 '식품첨가물',
 '알레르기',
 '방사능',
 '알레르기',
 '동식물질병',
 '방사능',
 '잔류농약',
 '영양성분',
 '잔류농약',
 '동식물질병',
 '미생물',
 '중금속',
 '위생',
 '건강',
 '미생물',
 '미생물',
 '식품첨가물',
 '안전',
 '원산지',
 '미생물',
 '알레르기',
 '식품첨가물',
 '식품첨가물',
 '식품첨가물',
 '기타',
 '동식물질병',
 '식품첨가물',
 '동식물질병',
 '잔류농약',
 '안전',
 '미생물',
 '알레르기',
 '미생물',
 '안전',
 '안전',
 '영양성분',
 '미생물',
 '미생물',
 '영양성분',
 '광고(허위, 과대 등)',
 '알레르기',
 '방사능',
 '미생물',
 '미생물',
 '의약품성분',
 '잔류농약',
 '식품첨가물',
 '건강',
 '미생물',
 '미생물',
 '안전',
 '동식물질병',
 '잔류농약',
 '알레르기',
 '안전',
 '잔류농약',
 '기구용기포장유래물질',
 '식품첨가물',
 '미생물',
 'GMO(LMO)',
 '기타',
 '중금속',
 '안전',
 '기타',
 '위생',
 '미생물',
 '위생',
 '동

In [102]:
tf['예측_원인요소'] = [predict_food_type(text) for text in tf['내용'].str[:1000]]

In [103]:
tf[['내용', '원인요소', '예측_원인요소']]

,내용,원인요소,예측_원인요소
7613,"광시장족자치구(广西壮族自治区) 시장감독관리국, 3대 식품(샘플 126건) 중 전분 ...",생물학적 위해요소>미생물>미생물일반,미생물
3341,"장쑤(江苏)성 시장감독관리국, 식품 샘플 806건 중 12건 부적합(식용농산물 2건...",생물학적 위해요소>미생물>대장균군,미생물
14529,- 통보일자: 28/04/2020- 통보번호: 2020.1794 - 통보국: 네덜란...,"생물학적 위해요소>곰팡이독소>아플라톡신(B1, B2, G1 및 G2)",곰팡이독소
19052,-수입상: Tea House Taipei(公悅企業有限公司)-수입상 주소: 타오위안(...,화학적 위해요소>잔류농약>티아크로프리드(Thiacloprid),잔류농약
3474,- 통보일자: 06/11/2020- 통보번호: 2020.4814 - 통보국: 불가리...,"생물학적 위해요소>곰팡이독소>아플라톡신(B1, B2, G1 및 G2)",곰팡이독소
...,...,...,...
17206,- 통보일자: 21/02/2020- 통보번호: 2020.0877 - 통보국: 독일-...,물리적 위해요소>이물질>플라스틱,이물질
2859,"일본 농림수산성은 11월 15일, 카가와현의 고병원성 조류인플루엔자 의사환축의 확인...",생물학적 위해요소>동식물질병>동물질병,동식물질병
1493,-제조사식별번호: 3010791553-제조사명: Samyang Foods Co Lt...,표시광고>알레르기>알레르기일반,알레르기
14644,"[중앙 계획 샘플검사] 푸젠(福建)성 시장감독관리국, 19대 식품(669건 샘플) ...",화학적 위해요소>기타>기타(Dimethyl Fumarate),미생물


In [107]:
sf = split_column(tf, '원인요소')
sf = sf[['내용', '예측_원인요소', '원인요소_2']]
sf

,내용,예측_원인요소,원인요소_2
7613,"광시장족자치구(广西壮族自治区) 시장감독관리국, 3대 식품(샘플 126건) 중 전분 ...",미생물,미생물
3341,"장쑤(江苏)성 시장감독관리국, 식품 샘플 806건 중 12건 부적합(식용농산물 2건...",미생물,미생물
14529,- 통보일자: 28/04/2020- 통보번호: 2020.1794 - 통보국: 네덜란...,곰팡이독소,곰팡이독소
19052,-수입상: Tea House Taipei(公悅企業有限公司)-수입상 주소: 타오위안(...,잔류농약,잔류농약
3474,- 통보일자: 06/11/2020- 통보번호: 2020.4814 - 통보국: 불가리...,곰팡이독소,곰팡이독소
...,...,...,...
17206,- 통보일자: 21/02/2020- 통보번호: 2020.0877 - 통보국: 독일-...,이물질,이물질
2859,"일본 농림수산성은 11월 15일, 카가와현의 고병원성 조류인플루엔자 의사환축의 확인...",동식물질병,동식물질병
1493,-제조사식별번호: 3010791553-제조사명: Samyang Foods Co Lt...,알레르기,알레르기
14644,"[중앙 계획 샘플검사] 푸젠(福建)성 시장감독관리국, 19대 식품(669건 샘플) ...",미생물,기타


In [111]:
sf.to_excel('실제_예측.xlsx')

In [110]:
# Calculate the percentage of exact matches between '원인요소' and '예측_원인요소'
exact_matches = (sf['원인요소_2'] == sf['예측_원인요소']).mean() * 100

exact_matches


82.1

In [112]:
y_true = sf['원인요소_2']
y_pred = sf['예측_원인요소']

In [113]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def plot_confusion_matrix_and_classification_report(y_true, y_pred, class_names=None):
    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Display classification report
    report = classification_report(y_true, y_pred, target_names=class_names)
    print("Classification Report:\n")
    print(report)

# Plot confusion matrix and classification report
plot_confusion_matrix_and_classification_report(y_true, y_pred)

ModuleNotFoundError: No module named 'matplotlib'

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Redefine the similarity functions
def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    return len(set1 & set2) / len(set1 | set2)

def cosine_sim(str1, str2):
    vectors = CountVectorizer().fit_transform([str1, str2]).toarray()
    return cosine_similarity(vectors)[0, 1]

# Apply the similarity measures to each row
tf['Jaccard Similarity'] = tf.apply(lambda row: jaccard_similarity(row['원인요소'], row['예측_원인요소']), axis=1)
tf['Cosine Similarity'] = tf.apply(lambda row: cosine_sim(row['원인요소'], row['예측_원인요소']), axis=1)

tf.head(2)


# Calculate the average similarity scores
average_jaccard = tf['Jaccard Similarity'].mean()
average_cosine = tf['Cosine Similarity'].mean()

average_jaccard, average_cosine


(0.4662416666666666, 0.5505557866223969)